making psf bias DV and contaminate simulated theory datavector <br>
based on code from here:  https://github.com/des-science/Y3_shearcat_tests/blob/212c03bfc239c9cff5419efbe70bdfdba28639ec/alpha-beta-eta-test/code/produce_2pcfpsfbias.py
 <br> and here: https://github.com/des-science/Y3_shearcat_tests/blob/master/alpha-beta-eta-test/forecast/contaminate.py

In [1]:
import h5py
import getdist
from getdist import plots, MCSamples
from astropy.io import fits
import itertools

import numpy as np

# Make Contaminant

In [4]:
#the relevant files will all be in the TXdata folders:
# des: '../desy3/TXdata/'
# hsc: '../hscy3/TXdata/'
# kids: '../KiDS/TXdata/'

#copy and paste the path you want here:
path= '../KiDS/TXdata/NEWruns/'

#DES and HSC have 4 bins, KiDS has 5.
nbins=5

In [6]:
#load rowes
with h5py.File(path+'rowe_stats-250JK-catms-9bin.hdf5') as f:
    meanr = f['rowe_statistics/rowe_0_PSF-reserved/theta'][:]
    rho0p = f['rowe_statistics/rowe_0_PSF-reserved/xi_plus'][:]
    rho0m = f['rowe_statistics/rowe_0_PSF-reserved/xi_minus'][:]
    rho1p = f['rowe_statistics/rowe_1_PSF-reserved/xi_plus'][:]
    rho1m = f['rowe_statistics/rowe_1_PSF-reserved/xi_minus'][:]
    rho2p = f['rowe_statistics/rowe_2_PSF-reserved/xi_plus'][:]
    rho2m = f['rowe_statistics/rowe_2_PSF-reserved/xi_minus'][:]
    rho3p = f['rowe_statistics/rowe_3_PSF-reserved/xi_plus'][:]
    rho3m = f['rowe_statistics/rowe_3_PSF-reserved/xi_minus'][:]
    rho4p = f['rowe_statistics/rowe_4_PSF-reserved/xi_plus'][:]
    rho4m = f['rowe_statistics/rowe_4_PSF-reserved/xi_minus'][:]
    rho5p = f['rowe_statistics/rowe_5_PSF-reserved/xi_plus'][:]
    rho5m = f['rowe_statistics/rowe_5_PSF-reserved/xi_minus'][:]

In [7]:
#load tau fitting chains
alist = []
blist = []
elist = []
with h5py.File(path+'tau_stats-250JK-catms-9bin.hdf5') as f:
    for n in range(nbins):
        alist.append(f[f'tau_statistics/chain_bestfits_PSF-reserved/bin_{n}/alpha'][:])
        blist.append(f[f'tau_statistics/chain_bestfits_PSF-reserved/bin_{n}/beta'][:])
        elist.append(f[f'tau_statistics/chain_bestfits_PSF-reserved/bin_{n}/eta'][:])

In [8]:
# for HSC, we're going to make rho0m all zeros since they're different lengths
# and cosmosis does not handle that correctly at the moment, so we're considering a xip-only inference 
#rho0m = rho1m = rho2m = rho3m = rho4m = rho5m = np.zeros(13)

a=[i for i in range(nbins)]
bin_pairs=[]
for p in itertools.combinations_with_replacement(a, 2): bin_pairs.append(p)
veclist = []
for z in range(len(alist[0])):
    dxip = [alist[i][z]*alist[j][z]*rho0p + blist[i][z]*blist[j][z]*rho1p + elist[i][z]*elist[j][z]*rho3p + (blist[i][z]*alist[j][z] + blist[j][z]*alist[i][z])*rho2p + (blist[i][z]*elist[j][z] + blist[j][z]*elist[i][z])*rho4p + (elist[i][z]*alist[j][z] +elist[j][z]*alist[i][z])*rho5p for i,j in bin_pairs]
    dxim = [alist[i][z]*alist[j][z]*rho0m + blist[i][z]*blist[j][z]*rho1m + elist[i][z]*elist[j][z]*rho3m + (blist[i][z]*alist[j][z] + blist[j][z]*alist[i][z])*rho2m + (blist[i][z]*elist[j][z] + blist[j][z]*elist[i][z])*rho4m + (elist[i][z]*alist[j][z] +elist[j][z]*alist[i][z])*rho5m for i,j in bin_pairs]
    dxi = dxip + dxim
    veclist.append(np.concatenate(np.c_[dxi]))
covmat = np.cov(np.c_[veclist].T)



In [10]:
al = []
bl = []
el = []
with h5py.File(path+'tau_stats-250JK-catms-9bin.hdf5') as f:
    for n in range(nbins):
        al.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/alpha']))
        bl.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/beta']))
        el.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/eta']))

In [ ]:
#load rowes
with h5py.File(path+'rowe_stats-interp.hdf5') as f:
    meanr = f['rowe_statistics/rowe_0_PSF-reserved/theta'][:]
    rho0p = f['rowe_statistics/rowe_0_PSF-reserved/xi_plus'][:]
    rho0m = f['rowe_statistics/rowe_0_PSF-reserved/xi_minus'][:]
    rho1p = f['rowe_statistics/rowe_1_PSF-reserved/xi_plus'][:]
    rho1m = f['rowe_statistics/rowe_1_PSF-reserved/xi_minus'][:]
    rho2p = f['rowe_statistics/rowe_2_PSF-reserved/xi_plus'][:]
    rho2m = f['rowe_statistics/rowe_2_PSF-reserved/xi_minus'][:]
    rho3p = f['rowe_statistics/rowe_3_PSF-reserved/xi_plus'][:]
    rho3m = f['rowe_statistics/rowe_3_PSF-reserved/xi_minus'][:]
    rho4p = f['rowe_statistics/rowe_4_PSF-reserved/xi_plus'][:]
    rho4m = f['rowe_statistics/rowe_4_PSF-reserved/xi_minus'][:]
    rho5p = f['rowe_statistics/rowe_5_PSF-reserved/xi_plus'][:]
    rho5m = f['rowe_statistics/rowe_5_PSF-reserved/xi_minus'][:]

In [11]:
#Writting final contamination
hdu = fits.PrimaryHDU()
hdul = fits.HDUList([hdu])

dxip_list = []; dxim_list = []; bin1_listp = []; bin2_listp = []; angbin_listp = []; ang_listp = [];
bin1_listm = []; bin2_listm = []; angbin_listm = []; ang_listm = []
covxip_list = []; covxim_list = []


for i,j in bin_pairs:
    dxip = al[i]*al[j]*rho0p + bl[i]*bl[j]*rho1p + (bl[i]*al[j] + bl[j]*al[i])*rho2p + el[i]*el[j]*rho3p + (bl[i]*el[j] + bl[j]*el[i])*rho4p + (el[i]*al[j] +el[j]*al[i])*rho5p
    dxim = al[i]*al[j]*rho0m + bl[i]*bl[j]*rho1m + (bl[i]*al[j] + bl[j]*al[i])*rho2m + el[i]*el[j]*rho3m + (bl[i]*el[j] + bl[j]*el[i])*rho4m + (el[i]*al[j] +el[j]*al[i])*rho5m

    ##COMMMENT FOR HSC:
    meanrp=meanrm=meanr
    
    
    ang_listp.append(meanrp)
    bin1_listp.append(np.array( [i + 1]*len(meanrp)))
    bin2_listp.append(np.array( [j + 1]*len(meanrp)))
    angbin_listp.append(np.arange(len(meanrp)))
    
    ang_listm.append(meanrm)
    bin1_listm.append(np.array( [i + 1]*len(meanrm)))
    bin2_listm.append(np.array( [j + 1]*len(meanrm)))
    angbin_listm.append(np.arange(len(meanrm)))
    
    dxip_list.append(dxip)
    dxim_list.append(dxim)

hdul.insert(1, fits.ImageHDU(covmat, name='COVMAT'))
bin1arrayp = np.concatenate(bin1_listp)
bin2arrayp = np.concatenate(bin2_listp)
angbinarrayp = np.concatenate(angbin_listp)
valuearray = np.concatenate(dxip_list)
angarrayp = np.concatenate(ang_listp)

bin1arraym = np.concatenate(bin1_listm)
bin2arraym = np.concatenate(bin2_listm)
angbinarraym = np.concatenate(angbin_listm)
angarraym = np.concatenate(ang_listm)

##Format of the fit file output
names=['BIN1', 'BIN2','ANGBIN', 'VALUE', 'ANG']
forms = ['i4', 'i4', 'i4',  'f4',  'f4']
dtype = dict(names = names, formats=forms)
nrows = len(angarrayp)
outdata = np.recarray((nrows, ), dtype=dtype)
array_list = [bin1arrayp, bin2arrayp, angbinarrayp, valuearray, angarrayp ]
for array, name in zip(array_list, names): outdata[name] = array 
corrhdu = fits.BinTableHDU(outdata, name='delta_xip')
hdul.insert(2, corrhdu)
nrows = len(angarraym)
outdata = np.recarray((nrows, ), dtype=dtype)
valuearray = np.concatenate(dxim_list)
array_list = [bin1arraym, bin2arraym, angbinarraym, valuearray, angarraym ]
for array, name in zip(array_list, names): outdata[name] = array 
corrhdu = fits.BinTableHDU(outdata, name='delta_xim')
hdul.insert(3, corrhdu)
hdul.writeto('NEWruns/KiDS-1000/KiDS-1000_contaminant_250JK-catms.fits', overwrite=True)
print('Written!')

Written!


# Contaminate DV

In [12]:
#the theory data vector files will all be in the cosmosis output folders:
# des: '../cosmosis-files/datavec/theory/DESsimulated_theoryDV.fits'
# hsc: '../cosmosis_files/datavec/theory/HSCsimulated_theoryDV.fits'
# kids: '../cosmosis_files/datavec/theory/KiDSsimulated_theoryDV.fits'

#copy and paste the path you want here:
dv= '../cosmosis_files/datavec/theory/KiDSsimulated_theoryDV.fits'

#DES and HSC have 4 bins, KiDS has 5.
nbins=5

In [15]:
def get_error(covmatrix, lengths, name):
    import numpy as np
    if name is not None:
        if (name=='xip'):
            start = 0
            end =start + lengths[0]
        elif (name=='xim'):
            start = lengths[0]
            end =start + lengths[1]
        elif (name=='gammat'):
            start = lengths[0] + lengths[1]
            end =start + lengths[2]
        elif (name=='wtheta'):
            start = lengths[0] + lengths[1]+ lengths[2]
            end =start + lengths[3]
        return np.diagonal(covmatrix)[start:end]**0.5
    else:
        print("Correlation function not defined")
        return None

import fitsio
covmatrixfit_ori=fitsio.read(dv,ext=1)
xipfit_ori=fitsio.read(dv,ext=2)
ximfit_ori=fitsio.read(dv,ext=3)

#read contaminant
covmatrixfit_cont=fitsio.read('NEWruns/KiDS-1000/KiDS-1000_contaminant_250JK-catms.fits',ext=1)
xipfit_cont=fitsio.read('NEWruns/KiDS-1000/KiDS-1000_contaminant_250JK-catms.fits',ext=2)
ximfit_cont=fitsio.read('NEWruns/KiDS-1000/KiDS-1000_contaminant_250JK-catms.fits',ext=3)
dxipbin = xipfit_cont['VALUE']
dximbin = ximfit_cont['VALUE']

nsig = 2
lengths = [len(xipfit_cont), len(ximfit_cont)]
# run both, uncomment which one you need 
#upper bound:
dxipbin += nsig*get_error(covmatrixfit_cont, lengths, 'xip')
dximbin += nsig*get_error(covmatrixfit_cont, lengths, 'xim')
#lower bound:
#dxipbin -= nsig*get_error(covmatrixfit_cont, lengths, 'xip')
#dximbin -= nsig*get_error(covmatrixfit_cont, lengths, 'xim')


a=[i for i in range(1,nbins+1)]
b=[j for j in range(1,nbins+1)]
bin_pairs=[]
for p in itertools.product(a, b):
    bin_pairs.append(p)
for i,j in bin_pairs:
    binp = (xipfit_ori['BIN1']==i)&(xipfit_ori['BIN2']==j)
    binm = (ximfit_ori['BIN1']==i)&(ximfit_ori['BIN2']==j)
    idxbinsp =  list(itertools.compress(range(len(binp)),  binp))
    idxbinsm =  list(itertools.compress(range(len(binm)),  binm))
    if (len(idxbinsp)!=0): xipfit_ori['VALUE'][binp] -=dxipbin[binp]
    if (len(idxbinsm)!=0): ximfit_ori['VALUE'][binm] -=dximbin[binm]

hdulist = fits.open(dv)
#delete all xip, xim but keep header
oldheaders =  [hdulist[2].header, hdulist[3].header]
hdulist.pop(index=2);
hdulist.pop(index=2);

xiphdu = fits.BinTableHDU(xipfit_ori)
ximhdu = fits.BinTableHDU(ximfit_ori)
hdulist.insert(2, xiphdu)
hdulist.insert(3, ximhdu)
hdulist[2].header = oldheaders[0]
hdulist[3].header = oldheaders[1]
hdulist.writeto('../cosmosis_files/datavec/contaminated/NEWruns/KiDS_1000/KiDS-1000_Contaminated_2sig_250JK_catms_sup.fits', overwrite=True)
print('written')


written
